In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
df = pd.read_csv("https://raw.githubusercontent.com/gmfv/YIO/main/datosfiltrados.csv",index_col=0,skiprows=0)

 **Seleccionamos las columnas a utilizar**

In [3]:
copy_df=df.copy()
#df = df.drop(df.columns[[0, 1, 2, 3, 5, 7, 12]], axis='columns')
#df.head()
copy_df=copy_df.set_index("id_anony")
copy_df=copy_df[copy_df["Asignatura"] == "CALCULO 2"]
columns_keep=["Cod.Car.Sec","Anho","Semestre", "Aprobado", "Anho.Firma",
              "Primer.Par", "Segundo.Par", "AOT"]
copy_df=copy_df[columns_keep]
copy_df#.head()

,Cod.Car.Sec,Anho,Semestre,Aprobado,Anho.Firma,Primer.Par,Segundo.Par,AOT
id_anony,,,,,,,,
es_1444,ELE-PLS13,2017,1,N,0,7,0,0
es_4238,CGF-PLS13,2017,1,N,0,5,0,0
es_4245,ELE-PLS13,2017,1,S,2016,0,0,0
es_4967,CIV-PLS13,2017,1,N,0,0,0,0
es_414,CGF-PLS13,2017,1,S,2017,21,10,0
...,...,...,...,...,...,...,...,...
es_9590,IND-PLS13,2019,2,N,0,5,4,2
es_9697,CIV-PLS13,2019,2,N,0,5,0,5
es_9437,CIV-PLS13,2019,2,N,0,0,0,0


**Reemplazamos las salidas (Si aprobo o no) por 1 o 0, las carreras por códigos**

In [4]:
copy_df['Aprobado']=copy_df['Aprobado'].replace(['N', 'S'],['0', '1'])
carreras=pd.Series(['0', '1', '2', '3', '4', '5', '6', '7', '8','9'],index=[copy_df['Cod.Car.Sec'].unique()])
#copy_df['Cod.Car.Sec']=copy_df['Cod.Car.Sec'].replace(copy_df['Cod.Car.Sec'].unique(),['0', '1', '2', '3', '4', '5', '6', '7', '8','9'])
copy_df['Cod.Car.Sec']=copy_df['Cod.Car.Sec'].replace(carreras)
#copy_df = copy_df.fillna(0)
copy_df#.head()
carreras

ELE-PLS13     0
CGF-PLS13     1
CIV-PLS13     2
MCT-PLS13     3
ECA-PLS13     4
IND-PLS13     5
MEC-PLS13     6
ELE-PLS09     7
IND-PLS09     8
CIV-PLS09     9
dtype: object

**Descartamos columnas para facilitar analisis inicial**

---
Solo tenemos en cuenta el rendimiento de primer, segundo parcial y taller


In [5]:
X = np.array(copy_df.drop(['Cod.Car.Sec', 'Anho','Semestre','Anho.Firma','Aprobado'], axis=1))
y = np.array(copy_df['Aprobado'])

In [6]:
model = linear_model.LogisticRegression()
model.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
predictions = model.predict(X)
print(predictions)

['0' '0' '0' ... '0' '0' '0']


Precision media de:

In [8]:
model.score(X,y)

0.8090737240075614

# **VALIDACION**

Dividimos nuestros datos en una seccion para entrenar y otra para validar

In [9]:
validation_size = 0.20
seed = 23
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)

Entrenamos a nuestro modelo con el 80% de los datos. Obteniendo una media de exactitud y desviación estandar que se imprimen en pantalla

In [10]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)

Logistic Regression: 0.814817 (0.032689)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [11]:
predictions = model.predict(X_validation)
print(accuracy_score(Y_validation, predictions))

0.8050314465408805


Tener en cuenta que la cantidad de datos reservados para la validacion era pequena en comparacion

# **Prueba** 

In [ ]:
X_new = pd.DataFrame({'Primer.Par': [15], 'Segundo.Par': [24], 'AOT': [4]})
model.predict(X_new)

array(['1'], dtype=object)

## **Comparación de precisión por carrera**

In [51]:
accuracy=[]
mc=np.zeros(4)
for i in range(7):
  datfriend = copy_df[copy_df['Cod.Car.Sec']== str(i)]

  if (i == 0):
    datfriend = pd.concat([datfriend, copy_df[copy_df['Cod.Car.Sec']== str(7)]], axis=0)
  elif (i == 2):
    datfriend = pd.concat([datfriend, copy_df[copy_df['Cod.Car.Sec']== str(9)]], axis=0)
  elif (i == 5):
    datfriend = pd.concat([datfriend, copy_df[copy_df['Cod.Car.Sec']== str(8)]], axis=0)

  X = np.array(datfriend.drop(['Cod.Car.Sec', 'Anho','Semestre','Anho.Firma','Aprobado'], axis=1))
  y = np.array(datfriend['Aprobado'])
  model = linear_model.LogisticRegression()
  model.fit(X,y)
  X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, y, test_size=0.5)
  kfold = model_selection.KFold(n_splits=10)
  cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
  predictions = model.predict(X_validation)
  accuracy.append(accuracy_score(Y_validation, predictions))
  matrizdeconfusion= confusion_matrix(Y_validation, predictions)
  matrizdeconfusion = matrizdeconfusion.flatten()   #Convertimos la matriz de confusion a 1 dimension 
  mc=np.vstack((mc,matrizdeconfusion)) #Colocamos al final de la matriz que recolecta los TT, FT, TF, FF de las carreras
accuracy = np.array(accuracy) #Convertimos la lista a un arreglos numpy, esto era porque queria intentar concater la matriz mc y el de accuracy, pero no pude todavia gg
mc=np.delete(mc, 0, axis=0) #Inicialmente la matriz mc se creo con ceros, que al final se eliminan, se podria encontrar la forma de obviar este paso, pero me daba error si no definia el tipo


**Convertimos los arreglos en Dataframes y concatenamos ambos**

In [57]:
datinha=pd.DataFrame(mc, index=['ELE', 'CGF', 'CIV', 'MCT', 'ECA', 'IND', 'MEC'], columns=['FF', 'FT', 'TF', 'TT'])
datinha2=pd.DataFrame(accuracy, index=['ELE', 'CGF', 'CIV', 'MCT', 'ECA', 'IND', 'MEC'], columns=['Accuracy'])
pd.concat([datinha, datinha2], axis=1)

,FF,FT,TF,TT,Accuracy
ELE,76.0,15.0,14.0,88.0,0.849741
CGF,27.0,0.0,2.0,4.0,0.939394
CIV,131.0,33.0,29.0,162.0,0.825352
MCT,30.0,6.0,6.0,47.0,0.865169
ECA,12.0,2.0,2.0,11.0,0.851852
IND,25.0,8.0,7.0,26.0,0.772727
MEC,12.0,5.0,2.0,14.0,0.787879
